In [4]:
library('tidyverse')
library('e1071')

In [5]:
dataset <- read.csv("mushroomdata/agaricus-lepiota.csv", header=FALSE, as.is=TRUE,
                               strip.white=TRUE, colClasses="character")

In [6]:
colnames(dataset) <- c("class", "cap_shape", "cap_surface", 
                        "cap_color", "bruises", "odor", 
                        "gill_attachement", "gill_spacing", "gill_size", 
                        "gill_color", "stalk_shape", "stalk_root", 
                        "stalk_surface_above_ring", "stalk_surface_below_ring", "stalk_color_above_ring", 
                        "stalk_color_below_ring", "veil_type", "veil_color", 
                        "ring_number", "ring_type", "spore_print_color", 
                        "population", "habitat")
dataset <- dataset %>% map_df(function(.x) as.factor(.x))
levels(dataset$class) <- c("edible", "poisonous")
levels(dataset$cap_shape) <- c("bell", "conical", "flat", "knobbed", "sunken", "convex")
levels(dataset$cap_color) <- c("buff", "cinnamon", "red", "gray", "brown", "pink", 
                                "green", "purple", "white", "yellow")
levels(dataset$cap_surface) <- c("fibrous", "grooves", "scaly", "smooth")
levels(dataset$bruises) <- c("no", "yes")
levels(dataset$odor) <- c("almond", "creosote", "foul", "anise", "musty", "none", "pungent", "spicy", "fishy")
levels(dataset$gill_attachement) <- c("attached", "free")
levels(dataset$gill_spacing) <- c("close", "crowded")
levels(dataset$gill_size) <- c("broad", "narrow")
levels(dataset$gill_color) <- c("buff", "red", "gray", "chocolate", "black", "brown", "orange", 
                                 "pink", "green", "purple", "white", "yellow")
levels(dataset$stalk_shape) <- c("enlarging", "tapering")
levels(dataset$stalk_root) <- c("missing", "bulbous", "club", "equal", "rooted")
levels(dataset$stalk_surface_above_ring) <- c("fibrous", "silky", "smooth", "scaly")
levels(dataset$stalk_surface_below_ring) <- c("fibrous", "silky", "smooth", "scaly")
levels(dataset$stalk_color_above_ring) <- c("buff", "cinnamon", "red", "gray", "brown", "pink", 
                                "green", "purple", "white", "yellow")
levels(dataset$stalk_color_below_ring) <- c("buff", "cinnamon", "red", "gray", "brown", "pink", 
                                "green", "purple", "white", "yellow")
levels(dataset$veil_type) <- "partial"
levels(dataset$veil_color) <- c("brown", "orange", "white", "yellow")
levels(dataset$ring_number) <- c("none", "one", "two")
levels(dataset$ring_type) <- c("evanescent", "flaring", "large", "none", "pendant")
levels(dataset$spore_print_color) <- c("buff", "chocolate", "black", "brown", "orange", 
                                        "green", "purple", "white", "yellow")
levels(dataset$population) <- c("abundant", "clustered", "numerous", "scattered", "several", "solitary")
levels(dataset$habitat) <- c("wood", "grasses", "leaves", "meadows", "paths", "urban", "waste")

dataset <- dataset %>% select(- veil_type)

In [7]:
set.seed(1810)
datasetsample <- caret::createDataPartition(y = dataset$class, times = 1, p = 0.1, list = FALSE)
train_dataset <- dataset[datasetsample, ]
test_dataset <- dataset[-datasetsample, ]

# Dane gotowe, budowa modelu i funkcji predykcji:

In [8]:
#function returning a list of trained models:
ensemble <- function(algorithm, formula, dataset, mcount=10, pattr=0.8, ...) {
  results <- list()
  for (i in 1:mcount) {
    attr_count <- max(1, floor(pattr * (ncol(dataset)-1))) # Of all columns, attr_count will be sampled.
    
    bootstrapped <- sample_n(dataset, nrow(dataset), replace=T)
        
    attr_randomized <- sample(bootstrapped[,-1], attr_count)
              
    attr_randomized$class <- bootstrapped$class
      
    new_model <- algorithm(formula, data=attr_randomized, ...)
    results[[i]] <- new_model
  }
  return(results)
}

In [9]:
# model_ens <- ensemble(algorithm=naive_bayes, formula=class~., dataset=train_dataset, mcount=100, pattr=0.5, usekernel=T, laplace=1)
model_ens <- ensemble(algorithm=svm, formula=class~., dataset=train_dataset, mcount=100, pattr=0.5, cost = 1000, gamma = 0.01)

In [37]:
# a function predicting the class probabilities for a given ensemble model and data
predict_fun <- function (ensemble_model, dataset, type="prob", single_predict=predict) {
    modelClasses <- levels(test_dataset$class)
    predictions <- matrix(0, nrow=nrow(dataset), ncol=length(modelClasses))
    colnames(predictions) <- modelClasses
    
    modelCount <- length(ensemble_model)
    
    for (i in 1:length(ensemble_model)) {
        newPrediction <- single_predict(ensemble_model[[i]], dataset, type="class")
        
        for (j in 1:length(newPrediction)) {
            predictions[j, newPrediction[j]] = predictions[j, newPrediction[j]] + 1/modelCount
        }
    }
    if (type == "class")
    {
        class_list <- c()
        for (i in 1:nrow(predictions)) {
            class_list[i] <- colnames(predictions)[which.max(predictions[i,])]
        }
        class_list <- as.factor(class_list)
        return(class_list)
    }
    return(predictions)
}

# Wyniki naszego ensemble modelu dla svm (e1071):

In [71]:
test_range <- 1:100
pred_prob <- predict_fun(model_ens, test_dataset[test_range,])
pred_class <- predict_fun(model_ens, test_dataset[test_range,], type="class")
result <- cbind(pred_prob, pred_class, as.data.frame(test_dataset[test_range,]$class))
colnames(result) <- c("EdibleProb", "PoisonousProb", "predicted class", "actual class")
result

EdibleProb,PoisonousProb,predicted class,actual class
<dbl>,<dbl>,<fct>,<fct>
0.03,0.97,poisonous,poisonous
0.99,0.01,edible,edible
0.99,0.01,edible,edible
0.06,0.94,poisonous,poisonous
1.00,0.00,edible,edible
1.00,0.00,edible,edible
1.00,0.00,edible,edible
1.00,0.00,edible,edible
1.00,0.00,edible,edible
